## test netcdf+

This is a more extensive integration test, if all the features of netcdf+ work as expected.

In [1]:
import openpathsampling as paths
from openpathsampling.netcdfplus import (
    NetCDFPlus, 
    ObjectStore, 
    StorableObject,
    NamedObjectStore,
    UniqueNamedObjectStore,
    DictStore,
    ImmutableDictStore,
    VariableStore,
    StorableNamedObject
)
from uuid import UUID
import numpy as np

In [2]:
class Node(StorableObject):
    def __init__(self, value):
        super(Node, self).__init__()
        self.value = value
        
    def __repr__(self):
        return 'Node(%s)' % self.value

In [3]:
class NamedNode(StorableNamedObject):
    def __init__(self, value):
        super(NamedNode, self).__init__()
        self.value = value
        
    def __repr__(self):
        return 'Node(%s)' % self.value

In [4]:
class NullNode(StorableObject):        
    _json_store_name = 'nullnodes'  # this should enable autocreation
    def __repr__(self):
        return 'NullNode()'

### Open new storage

try to create a new storage

In [5]:
st = NetCDFPlus('test_netcdfplus.nc', mode='w')

### Create some stores

In [6]:
class NodeIntStore(VariableStore):
    def __init__(self):
        super(NodeIntStore, self).__init__(Node, ['value'])
    
    def initialize(self):
        super(VariableStore, self).initialize()

        # Add here the stores to be imported
        self.create_variable('value', 'int')    

In [7]:
st.add_store('nodesnamed', NamedObjectStore(NamedNode))
st.add_store('nodesunique', UniqueNamedObjectStore(NamedNode))
st.add_store('dict', DictStore())
st.add_store('dictimmutable', ImmutableDictStore())
st.add_store('varstore', NodeIntStore())

And the default store. The last store for a particular object is used as the default if no specific store is specified.

In [8]:
st.add_class_json_store('nodes', Node)

store.nodes[Node]

Test automatic creation of store

In [9]:
ob = NullNode()

This should be storable and 

In [10]:
st.save(ob)

(store.nullnodes[NullNode], 6, 0)

In [11]:
print st.find_store(Node)
print st.find_store(NullNode)

store.nodes[Node]
store.nullnodes[NullNode]


Initialize the store

In [12]:
st.nodes.save(Node(10));

In [13]:
st.close()

### Reopen empty storage

In [14]:
st = NetCDFPlus('test_netcdfplus.nc', mode='a')

set caching of the new stores

In [15]:
for store in st.stores:
    store.set_caching(10)

Check if the stores were correctly loaded

In [16]:
assert('nodes' in st.objects)

In [17]:
assert('stores' in st.objects)

In [18]:
assert(len(st.nodes) == 1)

In [19]:
assert(len(st.stores) == 6)

AssertionError: 

In [ ]:
for store in st.stores:
    print '{:40} {:30}'.format(store, store.cache)

### Create variables types

Get a list of all possible variable types

In [ ]:
print sorted(st.get_var_types())

Make a dimension on length 2 to simplify dimension nameing.

Now we construct for each type a corresponding variable of dimensions 2x2x2.

In [ ]:
st.create_dimension('pair', 2)

In [ ]:
for var_type in st.get_var_types():
    st.create_variable(var_type, var_type, dimensions=('pair', 'pair', 'pair'))

In [ ]:
st.update_delegates()

In [ ]:
for var_name, var in sorted(st.variables.items()):
    print var_name, var.dimensions

In [ ]:
for var in sorted(st.vars):
    print var

#### Bool

In [ ]:
st.vars['bool'][:] = True

In [ ]:
print st.vars['bool'][:]

#### Float

In [ ]:
st.vars['float'][1,1] = 1.0

In [ ]:
print st.vars['float'][:]

#### Index
`Index` is special in the sense that it supports only integers that are non-negative. Negative values will be interpreted as `None`

In [ ]:
st.vars['index'][0,1,0] = 10
st.vars['index'][0,1,1] = -1
st.vars['index'][0,0] = None

In [ ]:
print st.vars['index'][0,1]
print st.vars['index'][0,0]

#### Int

In [ ]:
st.vars['int'][0,1,0] = 10
st.vars['int'][0,1,1] = -1

In [ ]:
print st.vars['int'][:]

#### JSON

The variable type JSON encode the given object as a JSON string in the shortest possible way. This includes using referenes to storable objects. 

In [ ]:
st.vars['json'][0,1,1] = {'Hallo': 2, 'Test': 3}

In [ ]:
st.vars['json'][0,1,0]

In [ ]:
st.vars['json'][0,1,0] = Node(10)

In [ ]:
print st.variables['json'][0,1,:]

All object types registered as being Storable by subclassing from `openpathsampling.base.StorableObject`.

#### JSONObj

A JSON serializable object. This can be normal very simple python objects, plus numpy arrays, and objects that implement `to_dict` and `from_dict`. This is almost the same as _JSON_ except if the object to be serialized is a storable object itself, it will not be referenced but the object itself will be turned into a JSON representation.

In [ ]:
nn = Node(10)
st.vars['jsonobj'][1,0,0] = nn

In [ ]:
print st.variables['jsonobj'][1,0,0]

#### Numpy

In [ ]:
st.vars['numpy.float32'][:] = np.ones((2,2,2)) * 3.0
st.vars['numpy.float32'][0] = np.ones((2,2)) * 7.0

In [ ]:
print st.vars['numpy.float32'][:]

#### Obj

You can store objects of a type which you have previously added. For loading you need to make sure that the class (and the store if set manually) is present when you load from the store.

In [ ]:
st.vars['obj.nodes'][0,0,0] = Node(1)
st.vars['obj.nodes'][0,1,0] = Node('Second')
st.vars['obj.nodes'][0,0,1] = Node('Third')

In some cases we can also assign one element to a group of elements like it is possible for numbers.

In [ ]:
st.vars['obj.nodes'][1] = Node(20)

In [ ]:
print st.variables['obj.nodes'][:]
print st.variables['nodes_json'][:]

In [ ]:
print st.vars['obj.nodes'][0,0,0]
print type(st.vars['obj.nodes'][0,0,0])

#### lazy

Lazy loading will reconstruct an object using proxies. These proxies behave almost like the loaded object, but will delay loading of the object until it is accessed. Saving for lazy objects is the same as for regular objects. Only loading return a proxy object.

In [ ]:
st.vars['lazyobj.nodes'][0,0,0] = Node('First')

The type of the returned object is `LoaderProxy` while the class is the actual class is the baseclass loaded by the store to not trigger loading when the `__class__` attribute is accessed. The actual object can be accessed by `__subject__` and doing so will trigger loading the object. All regular attributes will be delegated to `__subject__.attribute` and also trigger loading.

In [ ]:
proxy = st.vars['lazyobj.nodes'][0,0,0]
print 'Type:   ', type(proxy)
print 'Class:  ', proxy.__class__
print 'Content:', proxy.__subject__.__dict__
print 'Access: ', proxy.value

### Load/Save objects

Note that there are now 6 `Node` objects.

In [ ]:
print st.nodes[:]

In [ ]:
obj = Node('BlaBla')
st.nodes.save(obj);

Saving without specifying should use store `nodes` which was defined last.

In [ ]:
print len(st.nodes)
obj = Node('BlaBlaBla')
st.save(obj)
print len(st.nodes)

Get the index of the obj in the storage

In [ ]:
print st.idx(obj)

And test the different ways to access the contained `json`

#### 1. direct `json` using `variables` in the store

In [ ]:
print st.nodes.variables['json'][st.idx(obj)]

#### 2. direct `json` using `variables` in the full storage

In [ ]:
print st.variables['nodes_json'][st.idx(obj)]

#### 3. indirect `json` and reconstruct using `vars` in the store

In [ ]:
print st.nodes.vars['json'][st.idx(obj)]
print st.nodes.vars['json'][st.idx(obj)] is obj

#### 4. using the store accessor  `__getitem__` in the store

In [ ]:
print st.nodes[st.idx(obj)]
print st.nodes[st.idx(obj)] is obj

One importance difference is that a store like `nodes` has a cache (which we set to 10 before). Using `vars` will not use a store and hence create a new object!

### ObjectStores

ObjectStores are resposible to save and load objects. There are now 6 types available.

#### ObjectStore

The basic store which we have used before

#### NamedObjectStore

Supports to give objects names

In [ ]:
n = NamedNode(3)

NamedObjects have a `.name` property, which has a default.

In [ ]:
print n.name

and can be set.

In [ ]:
n.name = 'OneNode'
print n.name
n.name = 'MyNode'
print n.name

Once the object is saved, the name cannot be changed anymore.

In [ ]:
st.nodesnamed.save(n);

In [ ]:
try:
    n.name = 'NewName'
except ValueError as e:
    print '# We had an exception'
    print e
else:
    raise RuntimeWarning('This should have produced an error')

usually names are not unique (see next store). So we can have more than one object with the same name.

In [ ]:
n2 = NamedNode(9)
n2.name = 'MyNode'
st.nodesnamed.save(n2);

See the list of named objects

In [ ]:
print st.nodesnamed.name_idx

#### UniqueNamedObjectStore

The forces names to be unique

In [ ]:
st.nodesunique.save(n);

Note here that an object can be store more than once in a storage, but only if more than one store supports the file type.

In [ ]:
try:
    st.nodesunique.save(n2)
except RuntimeWarning as e:
    print '# We had an exception'
    print e
else:
    raise RuntimeWarning('This should have produced an error')


As said before this can only happen if you have more than one store for the same object type.

In [ ]:
print st.nodesunique.name_idx

some more tests. First saving onnamed objects. This is okay. Only given names should be unique.

In [ ]:
n3 = NamedNode(10)
n4 = NamedNode(12)
st.nodesunique.save(n3);
st.nodesunique.save(n4);

In [ ]:
n5 = NamedNode(1)
n5.name = 'MyNode'

In [ ]:
try:
    st.nodesunique.save(n5)
except RuntimeWarning as e:
    print '# We had an exception'
    print e
else:
    raise RuntimeWarning('This should have produced an error')

This works since it does a rename before saving.

In [ ]:
st.nodesunique.save(n5, 'NextNode');

In [ ]:
n6 = NamedNode(1)
n6.name = 'SecondNode'

In [ ]:
try:
    st.nodesunique.save(n6, 'MyNode')
except RuntimeWarning as e:
    print '# We had an exception'
    print e
else:
    raise RuntimeWarning('This should have produced an error')

#### DictStore

A dictstore works a like a dictionary on disk. The content is returned using `dict()`

In [ ]:
print dict(st.dict)
print st.dict.name_idx

In [ ]:
n1 = NamedNode(1)
n2 = NamedNode(2)
n3 = NamedNode(3)
st.dict['Number1'] = n1

In [ ]:
for key in sorted(st.dict):
    obj = st.dict[key]
    idxs = sorted(st.dict.name_idx[key])
    print key, ':', str(obj), idxs

In [ ]:
st.dict['Number2'] = n2

In [ ]:
for key in sorted(st.dict):
    obj = st.dict[key]
    idxs = sorted(st.dict.name_idx[key])
    print key, ':', str(obj), idxs

In [ ]:
st.dict['Number1'] = n3

In [ ]:
for key in sorted(st.dict):
    obj = st.dict[key]
    idxs = sorted(st.dict.name_idx[key])
    print key, ':', str(obj), idxs

In [ ]:
print st.dict['Number1']

In [ ]:
print st.dict.find('Number1')

In [ ]:
print '[', ', '.join(st.dict.variables['json'][:]), ']'

In [ ]:
for key in sorted(st.dict):
    obj = st.dict[key]
    idxs = sorted(st.dict.name_idx[key])
    print key, ':', str(obj), idxs

#### ImmutableDictStore

This adds the check that already used names cannot be used again

In [ ]:
try:
    st.dictimmutable['Number1'] = n1
    st.dictimmutable['Number1'] = n2
except RuntimeWarning as e:
    print '# We had an exception'
    print e
else:
    raise RuntimeWarning('This should have produced an error')

#### VariableStore

Store a node with an int as we defined for our `VariableStore`

In [ ]:
a = Node(30)
st.varstore.save(a);

clear the cache

In [ ]:
st.varstore.clear_cache()

And try loading

In [ ]:
assert(st.varstore[0].value == 30)

Try storing non int() parseable value

In [ ]:
try:
    a = Node('test')
    print st.varstore.save(a)
except ValueError as e:
    print '# We had an exception'
    print e
else:
    raise RuntimeWarning('This should have produced an error')